# Compare container dwell time distribution with results

In [ ]:
import os
import sys

import matplotlib.pyplot as plt
import numpy as np

path_to_conflowgen = os.path.abspath(
    os.path.join(
        os.pardir,  # notebooks
        os.pardir,  # tests
        os.pardir  # conflowgen
    )
)

path_to_conflowgen

Load previously defined data.

In [ ]:
sys.path.insert(
    0,
    path_to_conflowgen
)

In [ ]:
import conflowgen

conflowgen.__file__

In [ ]:
conflowgen.setup_logger(
    logging_directory="./data/logger",  # use subdirectory relative to Jupyter Notebook
)

database_chooser = conflowgen.DatabaseChooser(
    sqlite_databases_directory="../../data/databases"  # use subdirectory relative to Jupyter Notebook
)

Please make sure that you have previously run this demo file:

```bash
python ./examples/Python_Script/demo_continental_gateway.py
```

Only then, the following database entry should be available.

In [ ]:
file_name = "demo_continental_gateway.sqlite"

database_chooser.load_existing_sqlite_database(file_name)

In [ ]:
container_dwell_time_analysis_report = conflowgen.ContainerDwellTimeAnalysisReport()
container_dwell_time_analysis = container_dwell_time_analysis_report.analysis

## Container dwell time of standard containers

In [ ]:
container_dwell_time_distribution_manager = (
    conflowgen.ContainerDwellTimeDistributionManager()
)
distributions = (
    container_dwell_time_distribution_manager.get_container_dwell_time_distribution()
)

In [ ]:
for inbound_vehicle_type in set(conflowgen.ModeOfTransport):
    for outbound_vehicle_type in set(conflowgen.ModeOfTransport):
        container_dwell_times = container_dwell_time_analysis.get_container_dwell_times(
            container_delivered_by_vehicle_type=inbound_vehicle_type,
            container_picked_up_by_vehicle_type=outbound_vehicle_type,
            storage_requirement=conflowgen.StorageRequirement.standard,
        )
        if not container_dwell_times:
            print(f"No traffic from {inbound_vehicle_type} to {outbound_vehicle_type}")
            continue

        distribution = distributions[inbound_vehicle_type][outbound_vehicle_type][
            conflowgen.StorageRequirement.standard
        ]

        print(
            f"Traffic from {inbound_vehicle_type} to {outbound_vehicle_type}:",
            distribution,
        )

        fig, ax = plt.subplots()
        ax1 = container_dwell_time_analysis_report.get_report_as_graph(
            container_delivered_by_vehicle_type=inbound_vehicle_type,
            container_picked_up_by_vehicle_type=outbound_vehicle_type,
            storage_requirement=conflowgen.StorageRequirement.standard,
        )

        plt.axvline(distribution.minimum, color="dimgray")
        plt.axvline(distribution.maximum, color="dimgray")
        x = np.linspace(0, int(distribution.maximum), int(distribution.maximum))

        x_in_range = x[
            np.where((distribution.minimum < x) & (x < distribution.maximum))
        ]

        ax2 = ax1.twinx()

        ax2.plot(
            x_in_range,
            distribution.get_probabilities(x_in_range),
            color="gray",
            lw=5,
            alpha=0.9,
        )
        plt.show()

In [ ]:
text = container_dwell_time_analysis_report.get_report_as_text(
    container_delivered_by_vehicle_type=conflowgen.ModeOfTransport.truck,
    container_picked_up_by_vehicle_type=conflowgen.ModeOfTransport.feeder,
    storage_requirement=conflowgen.StorageRequirement.standard,
)
print(text)

In [ ]:
distribution_truck_to_feeder = distributions[conflowgen.ModeOfTransport.truck][
    conflowgen.ModeOfTransport.feeder
][conflowgen.StorageRequirement.standard]

distribution_truck_to_feeder

In [ ]:
text = container_dwell_time_analysis_report.get_report_as_text(
    container_delivered_by_vehicle_type=conflowgen.ModeOfTransport.feeder,
    container_picked_up_by_vehicle_type=conflowgen.ModeOfTransport.truck,
    storage_requirement=conflowgen.StorageRequirement.standard,
)
print(text)

In [ ]:
distribution_feeder_to_truck = distributions[conflowgen.ModeOfTransport.feeder][
    conflowgen.ModeOfTransport.truck
][conflowgen.StorageRequirement.standard]

distribution_feeder_to_truck

In [ ]:
database_chooser.close_current_connection()